In [1]:
import time

# Calculating the Relative Profits of all the columns

In [2]:
def zj_cj(l, cj):
    a = [0, 0]
    for i in range(2, len(cj)):
        a.append(-cj[i])
        
    for i in range(len(l)):
        for j in range(2, len(cj)):
            a[j] += l[i][j]*l[i][0]
    return a.index(min(a)), min(a)

# Calculating the ratio of the selected column and finding the minimum

In [3]:
def min_ratio(l, cj):
    m = 10000000000
    a = list()
    b = zj_cj(l, cj)[0]
    for i in range(len(l)):
        if l[i][b] > 0:
            a.append([l[i][1]/l[i][b], l[i][0]])
        else:
            a.append([m, l[i][0]])
    x = a[0][0]
    y = a[0][1]
    z = 0
    for i in range(1, len(l)):
        if x > a[i][0]:
            x = a[i][0]
            y = a[i][1]
            z = i
        elif x == a[i][0]:
            if y > a[i][1]:
                y = a[i][1]
                z = i
    return z

# Used Simplex Big-M method to optimize the matrix and finding the optimised cost using summation of CbXb

In [4]:
def optimize(l, cj):
    start_time = time.time()
    n_ = 0
    while(zj_cj(l, cj)[1] < 0):
        n_ += 1
        x_ = 0
        a = zj_cj(l, cj)[0]
        b = min_ratio(l, cj)
        for i in range(len(l[0])):
            l[b][i] = l[b][i]/l[b][a]
        l[b][0] = cj[a]
        for i in range(len(l)):
            if (l[i][a] != 0 and i != b):
                y = l[i][a]
                for j in range(1, len(l[0])):
                    l[i][j] = l[i][j] - (y*l[b][j])
        for i in range(len(l)):
            x_ += l[i][0]*l[i][1]
    x_ = str(x_)
    print("--- %s seconds ---" % (time.time() - start_time))
    return "Optimized Cost = " + x_, l

# All Functions of Simplex Big-M combined

In [6]:
def Simplex(S, D, dist):  
    l = list()
    for i in range(len(S) + len(D)):
        l.append([])

    M = 10000000000
    c = 1

    cj = [0, 0]                              #Constructing the Cj matrix which represents the 
    for i in range(len(S)):                  #coefficients of the variables in the objective function
        for j in range(len(D)):               
            cj.append(c*dist[i][j])
    for i in range(len(S) + len(D)):
        cj.append(0)
    for i in range(len(D)):
        cj.append(-M)
        
    for i in range(len(S) + len(D)):        #Constructing the Simplex Big-M Matrix till line 58
        if (i < len(S)):
            l[i].append(0)
            l[i].append(S[i])
            for j in range(len(S)*len(D)):
                if (i == j//len(D)):
                    l[i].append(1)
                else:
                    l[i].append(0)
                    
            for j in range(len(S) + len(D)):
                if (i == j):
                    l[i].append(1)
                else:
                    l[i].append(0)
            
            for j in range(len(D)):
                l[i].append(0)
                
        else:
            l[i].append(-M)
            l[i].append(D[i - len(S)])
            
            for j in range(len(S)):
                for k in range(len(D)):
                    if (k == (i - len(S))):
                        l[i].append(1)
                    else:
                        l[i].append(0)
                        
            for j in range(len(S) + len(D)):
                if (i == j):
                    l[i].append(-1)
                else:
                    l[i].append(0)
            
            for j in range(len(D)):
                if ((i - len(S)) == j):
                    l[i].append(1)
                else:
                    l[i].append(0)
    return optimize(l, cj)                        #Optimising the matrix to achieve the optimised transport cost

## Test Example 1

In [8]:
S = [3, 6]
D = [2, 4]
Dist = [[1, 3], [2, 5]]
print(Simplex(S, D, Dist))

--- 0.0003249645233154297 seconds ---
('Optimized Cost = 35.0', [[3, 1.0, 0.0, 1.0, -1.0, 0.0, 1.0, 0.0, 1.0, 0.0, -1.0, 0.0], [0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0], [1, 2.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0], [5, 6.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]])


## Test Example 2

In [7]:
S = [8, 4, 12, 6]
D = [3, 5, 10, 7, 5]
dist = [[1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8]]
print(Simplex(S, D, dist))

--- 0.0023012161254882812 seconds ---
('Optimized Cost = 142.0', [[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [3, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0], [5, 6.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0], [7, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0, 1.0, 1.0, 1.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, -1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0], [1, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [2,

## Example mentioned in Scope for Improvements

In [9]:
S = [8,4,12,6]
D = [3, 5, 10, 7, 5]
dist = [[5, 3, 2.236068, 3.60555, 5.830952], 
        [2.236068, 2.236068, 3, 5, 2], 
        [3.60555, 3, 4.1231, 5, 3.162278], 
        [3.162278, 2.82843, 1.41421, 5.09902, 3.60555]]
print(Simplex(S, D, dist))

--- 0.0022940635681152344 seconds ---
('Optimized Cost = 135.16811', [[3, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0], [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [3, 2.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0], [5.09902, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [5, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.